In [1]:
import sys
import numpy as np
from siman import header
from siman.header      import db # database dictionary
from siman.SSHTools    import SSHTools
from siman.calc_manage import smart_structure_read, add, res
from siman.geo         import supercell
from siman.database    import read_database, write_database

read_database() # read saved results

({},
 {'static': <siman.set_functions.InputSet at 0x7f9fd49ab710>,
  'opt': <siman.set_functions.InputSet at 0x7f9fd49ab780>},
 56)

## 0. Configure Siman (Can be done in ~/simanrc.py or ./project_conf.py)

In [2]:
#Instead of passwordless setup or for Windows you can use ssh_object (paramiko module is used):
user = "d.aksenov"
host = "10.30.16.168"
header.ssh_object = SSHTools()
header.ssh_object.setup(user=user, host=host, pkey="/home/aksenov/.ssh/id_rsa")
header.PATH2POTENTIALS = 'potcars/' # path to POTCAR files

## 1. Read structure and create supercell

In [3]:
st = smart_structure_read(input_geo_file = 'Li2/POSCAR') # read poscar
sc = supercell(st, [10,10,10]) # create 3x3x3 supercell, sizes close to 10 10 10 A 


-- Calculating mul_matrix for ortho: [10, 10, 10] 

mul_matrix_float:
 [[2.91205591 0.         0.        ]
 [0.         2.91205591 0.        ]
 [0.         0.         2.91205591]] 
mul_matrix:
 [[3 0 0]
 [0 3 0]
 [0 0 3]] 
Old vectors (rprimd):
 [[3.4 0.  0. ]
 [0.  3.4 0. ]
 [0.  0.  3.4]] 
New vectors (rprimd) of supercell:
 [[10.3  0.   0. ]
 [ 0.  10.3  0. ]
 [ 0.   0.  10.3]] 
The supercell should contain 53.999999999999986 atoms ... 
  -- OK 



## 2. Create vacancy and interstitial defects

In [4]:
sc_vac = sc.del_atom(25) # create vacancy defect in supercell
sc_oct = sc.add_atom([1/2 / 3, 1/2 / 3, 1 / 3], 'Li')  # create interstitial defect in octahedral position
sc_tet = sc.add_atom([1/4 / 3, 1/2 / 3, 1 / 3], 'Li')  # create interstitial defect in tetrahedral position

# sc_vac.write_poscar('Li_vac/POSCAR') # write POSCAR to check geometry
# sc_oct.write_poscar('Li_oct/POSCAR') 
# sc_tet.write_poscar('Li_tet/POSCAR')  

## 3 Run calculations 

In [5]:
if 1:
    it_folder = 'recitationCD/'
    add('Li333',    'opt', 1, up = 'up2', input_st = sc,     it_folder = it_folder, run = 0, ) 
    add('Li333vac', 'opt', 1, up = 'up2', input_st = sc_vac, it_folder = it_folder, run = 0) 
    add('Li333tet', 'opt', 1, up = 'up2', input_st = sc_tet, it_folder = it_folder, run = 0) 
    add('Li333oct', 'opt', 1, up = 'up2', input_st = sc_oct, it_folder = it_folder, run = 0) 

-- Attention!, cluster None is not found, using default CEE 

-- check_kpoints(): Kpoint   mesh is:  [4, 4, 4] 

-- check_kpoints(): The actual k-spacings are  [0.15 0.15 0.15] 

-- POSCAR was written to /home/aksenov/recitations/Rec10_DS/recitationCD///Li333.opt/1.POSCAR 

-- Attention! ngkpt =  [4, 4, 4]  is adopted from struct_des which you provided for it  Li333  and kspacing =  0.2 


Calculation ('Li333', 'opt', 1) successfully created

 

-- Attention!, cluster None is not found, using default CEE 

-- check_kpoints(): Kpoint   mesh is:  [4, 4, 4] 

-- check_kpoints(): The actual k-spacings are  [0.15 0.15 0.15] 

-- POSCAR was written to /home/aksenov/recitations/Rec10_DS/recitationCD///Li333vac.opt/1.POSCAR 

-- Attention! ngkpt =  [4, 4, 4]  is adopted from struct_des which you provided for it  Li333vac  and kspacing =  0.2 


Calculation ('Li333vac', 'opt', 1) successfully created

 

-- Attention!, cluster None is not found, using default CEE 

-- check_kpoints(): Kpoint   

## 4. Read results

In [6]:
header.check_job = 0
res('Li333',    'opt', 1, up = 'up1', show = 'fo') # try show = 'en', 'conv', 'est'
res('Li333vac', 'opt', 1, up = 'up1', show = 'fo')
res('Li333tet', 'opt', 1, up = 'up1',show = 'fo')
res('Li333oct', 'opt', 1, up = 'up1',show = 'fo')



Max. F. tot  (meV/A) = 
[0]; 

-- name                          |  energy(eV)|    Vector lenghts (A)   | Stresses (MPa)     | N MD, N SCF    
-- db['Li333.opt.1']             | -102.3599  |10.30;10.30;10.30|     39,39,39       |   1,10, 10    


Max. F. tot  (meV/A) = 
[86 82 49 51 51 50 48 45 42 38 32 22 21 18 16 13 14 15 14 13]; 

-- name                          |  energy(eV)|    Vector lenghts (A)   | Stresses (MPa)     | N MD, N SCF    
-- db['Li333vac.opt.1']          |  -99.9640  |10.30;10.30;10.30|    -89,-89,-89     |  20, 4, 86    


Max. F. tot  (meV/A) = 
[1865 1404  359  207  199  163  123   91   93   85   74   58   47   34
   32   33   35   33   27   21]; 

-- name                          |  energy(eV)|    Vector lenghts (A)   | Stresses (MPa)     | N MD, N SCF    
-- db['Li333tet.opt.1']          | -103.4905  |10.30;10.30;10.30|    389,358,358     |  20, 5,109    


Max. F. tot  (meV/A) = 
[2599 1950  352  164  156  128  111  132  125  102   77   46   38   38
   35   

(["db['Li333oct.opt.1']          | -103.3693  |10.30;10.30;10.30|    314,314,496     |  20, 5,109   "],
 [])

## 5. Write database to save results.

In [7]:
write_database()


Database has been successfully updated



## 6. Calculate defect energies

In [8]:
Li333 = db['Li333', 'opt', 1] # short name for calculation
Livac = db['Li333vac', 'opt', 1]
Litet = db['Li333tet', 'opt', 1]
Lioct = db['Li333oct', 'opt', 1]


#1 Point defects formation energy
natom = Li333.end.natom
print('Vacancy formation energy = {:3.3f} eV, E(relax) = {:3.3f} eV'.format(  
    Livac.e0 - Li333.e0 * (natom-1)/natom,    Livac.list_e_sigma0[0] - Livac.e0   )    )

print('Tet int formation energy = {:3.3f} eV, E(relax) = {:3.3f} eV'.format(  
    Litet.e0 - Li333.e0 * (natom+1)/natom,   Litet.list_e_sigma0[0] - Litet.e0   )    )

print('Oct int formation energy = {:3.3f} eV, E(relax) = {:3.3f} eV'.format(  Lioct.e0 - Li333.e0 * (natom+1)/natom,
    Lioct.list_e_sigma0[0] - Lioct.e0   )    )

Vacancy formation energy = 0.500 eV, E(relax) = 0.093 eV
Tet int formation energy = 0.765 eV, E(relax) = 1.870 eV
Oct int formation energy = 0.886 eV, E(relax) = 1.822 eV
